In [1]:
import os
import open3d as o3d
import numpy as np
import glob

In [2]:
data_root = "data/ModelResource_RigNetv1_preproccessed"
obj_folder = f'{data_root}/obj'
rig_folder = f'{data_root}/rig_info'

In [3]:
def get_mesh_joint_pair(obj_index, obj_folder, rig_folder):
    
    obj_path = f'{obj_folder}/{obj_index}.obj'
    rig_path = f'{rig_folder}/{obj_index}.txt'

    # Load and pre-process mesh
    mesh = o3d.io.read_triangle_mesh(obj_path, enable_post_processing=True)

    verts = np.asarray(mesh.vertices)
    centroid = verts.mean(axis=0)
    mesh.translate(-centroid)

    # extraxt join locations
    # disregard bone info. Only want joint locations
    joints = []
    with open(rig_path, "r") as f:
        tokens = f.readline().split()
        while(tokens[0] == "joints"):
            joints.append(list(map(float, tokens[2:])))
            tokens = f.readline().split()

    # Return joints as point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(np.asarray(joints))
    pcd.translate(-centroid)

    return mesh, pcd

In [4]:
def process_and_visualize(mesh: o3d.geometry.TriangleMesh, joints: o3d.geometry.PointCloud):

    # Get verices and centroid
    verts = np.asarray(mesh.vertices)
    centroid = verts.mean(axis=0)   

    # Mesh frame
    mesh_frame = o3d.geometry.LineSet.create_from_triangle_mesh(mesh)

    # Draw joints
    spheres = []
    for (x, y, z) in np.asarray(joints.points):
        sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.01)
        sphere.translate((x, y, z))
        sphere.paint_uniform_color([0, 1, 1])
        spheres.append(sphere)

    # Compute AABB and its longest side
    aabb = mesh.get_axis_aligned_bounding_box()
    min_bound = aabb.min_bound  # [x_min, y_min, z_min]
    max_bound = aabb.max_bound  # [x_max, y_max, z_max]
    lengths = max_bound - min_bound  # [Lx, Ly, Lz]
    longest = lengths.max()

    # Create a box for visualization
    box = o3d.geometry.LineSet.create_from_axis_aligned_bounding_box(aabb)
    box.translate(-centroid) 
    box.colors = o3d.utility.Vector3dVector([[1, 0, 0] for _ in box.lines]) 

    # Axes
    axes = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=longest * 0.5,
        origin=[0, 0, 0]
    )

    o3d.visualization.draw_geometries([mesh_frame, *spheres, box, axes],
                                      window_name="Mesh + AABB",
                                      width=800, height=600)

In [7]:
mesh, joints = get_mesh_joint_pair(341, obj_folder, rig_folder)
mesh, joints

(TriangleMesh with 1623 points and 3172 triangles., PointCloud with 43 points.)

In [ ]:
process_and_visualize(mesh, joints)

: 